# LangGraph + MCP + Ollama
## PLAN → USER LOOP → ACT Agent

- User can refine plan infinitely
- ACT only runs after approval
- Real file read / write example


In [ ]:
!pip install -U langgraph langchain langchain-ollama mcp

In [ ]:
from typing import TypedDict, List, Dict
from langgraph.graph import StateGraph, END
from langchain_ollama import ChatOllama
import os, json

In [ ]:
llm = ChatOllama(model="gpt-oss:20b", temperature=0)

## Agent State

In [ ]:
class AgentState(TypedDict):
    goal: str
    plan: List[str]
    messages: List[Dict]
    done: bool

## MCP Filesystem Tools

In [ ]:
from mcp.server.fastmcp import FastMCP

BASE_DIR = "/tmp/mcp_workspace"
os.makedirs(BASE_DIR, exist_ok=True)

mcp = FastMCP("filesystem")

@mcp.tool()
def read_file(path: str) -> str:
    return open(os.path.join(BASE_DIR, path)).read()

@mcp.tool()
def write_file(path: str, content: str) -> str:
    with open(os.path.join(BASE_DIR, path), "w") as f:
        f.write(content)
    return "ok"

In [ ]:
def call_tool(name, args):
    return mcp._tools[name](**args)

## PLAN NODE

In [ ]:
def planner_node(state: AgentState) -> AgentState:
    prompt = f"""
You are a planner.

User request:
{state['goal']}

Existing plan:
{state['plan']}

Rules:
- If no plan, create one
- If plan exists, modify it
- No execution
- Output numbered steps only
"""

text = llm.invoke(prompt).content
state['plan'] = [l.strip() for l in text.splitlines() if l.strip()]
state['done'] = False
return state

## EXECUTOR NODE

In [ ]:
def executor_node(state: AgentState) -> AgentState:
    prompt = f"""
You are an executor.

Plan:
{state['plan']}

History:
{state['messages']}

Return JSON only:
{{"tool": "read_file | write_file | done", "args": {{}}}}
"""

action = json.loads(llm.invoke(prompt).content)

if action['tool'] == 'done':
    state['done'] = True
    return state

state['messages'].append(action)
return state

In [ ]:
def tool_node(state: AgentState) -> AgentState:
    action = state['messages'][-1]
    result = call_tool(action['tool'], action.get('args', {}))
    state['messages'].append({'observation': result})
    return state

## Build PLAN Graph

In [ ]:
plan_graph = StateGraph(AgentState)
plan_graph.add_node("planner", planner_node)
plan_graph.set_entry_point("planner")
plan_graph.add_edge("planner", END)
plan_app = plan_graph.compile()

## Build ACT Graph

In [ ]:
act_graph = StateGraph(AgentState)
act_graph.add_node("executor", executor_node)
act_graph.add_node("tool", tool_node)
act_graph.set_entry_point("executor")
act_graph.add_conditional_edges(
    "executor",
    lambda s: END if s['done'] else 'tool'
)
act_graph.add_edge("tool", "executor")
act_app = act_graph.compile()

## PLAN MODE

In [ ]:
state = {
    'goal': 'Read x1.txt and x2.txt and write combined summary to summary.txt',
    'plan': [],
    'messages': [],
    'done': False
}

state = plan_app.invoke(state)
state['plan']

## USER MODIFIES PLAN

In [ ]:
state['goal'] = 'Change summary.txt to summ.txt'
state = plan_app.invoke(state)
state['plan']

## ACT MODE

In [ ]:
state = act_app.invoke(state)
state['messages']